<a href="https://colab.research.google.com/github/rohailkhan/deep_learning/blob/master/0504_Time_Invariant_Face_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# بسم الله الرحمن الرحيم
اللهم صل على محمد وعلى آل محمد كما صليت على إبراهيم وعلى آل إبراهيم إنك حميد مجيد، اللهم بارك على محمد وعلى آل محمد كما باركت على إبراهيم وعلى آل إبراهيم إنك حميد مجيد




In [2]:
from keras.models import Sequential
"""Import from keras_preprocessing not from keras.preprocessing, 
because Keras may or maynot contain the features discussed here 
depending upon when you read this article, until the keras_
preprocessed library is updated in Keras use the github version."""
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
import pandas as pd
import numpy as np

Using TensorFlow backend.


In [0]:
import os ,PIL

Convert Images to black and white

In [0]:
'''
source='E:\\Time-Invariant-Aging-Model-master\\fgsmallbw'
list=os.listdir(source)
for i in list:
    PIL.Image.open(i).convert('L').save(i)

In [0]:
#making the current working directory
os.chdir('/content/drive/My Drive/Aging model')

In [0]:
source='/content/drive/My Drive/Aging model/'

In [0]:
os.chdir(source)

In [0]:
# Put all files into a single dataframe
list_dir = [i for i in os.listdir('datafiles')]

In [0]:
# Put all files into a single dataframe
list_dir = [i for i in os.listdir('datafiles')]
final_df = pd.DataFrame(columns=['ID','Facial Index','Mandibular Index','Intercantal Index','Orbital Width Index','Eye Fissure Index','Vermilion Height Index','Mouth Face Width Index','AGE'])
interim_df = pd.DataFrame(columns=['ID','Facial Index','Mandibular Index','Intercantal Index','Orbital Width Index','Eye Fissure Index','Vermilion Height Index','Mouth Face Width Index','AGE'])
age_list=list()
for d in list_dir:
    interim_df.at[0,  ['Facial Index','Mandibular Index','Intercantal Index','Orbital Width Index','Eye Fissure Index','Vermilion Height Index','Mouth Face Width Index']] = pd.read_csv('datafiles/'+d, sep=" ", header=None).values[0][:7]
    interim_df['AGE'] = int(d[4:].replace('.txt','').replace('a','').replace('b',''))
    interim_df['ID'] = int(d[:3])
    final_df = pd.concat([final_df,interim_df])
    
# sort values by age of same person
final_df = final_df.sort_values(['ID','AGE'],ascending=True)

Saving in CSV

In [0]:
image_names=[]
for i in os.listdir('/content/drive/My Drive/Aging model/fgbw'):
  image_names.append(i)

In [0]:
image_names=sorted(image_names)

In [0]:
final_df['file_name']=image_names

In [0]:
final_df.to_csv('final_df.csv')

# Splitting data in training and validation(test) sets

In [0]:
final_df=pd.read_csv('/content/drive/My Drive/Aging model/final_df.csv')

In [5]:
final_df.shape

(1002, 11)

In [0]:
train_size = len(final_df['ID'].unique()) * .6

In [0]:
train = pd.DataFrame()
valid = pd.DataFrame()
for n,i in enumerate(final_df['ID'].unique()):
    if n< train_size:
        train = pd.concat([train, final_df[final_df['ID']==i]])
    else:
        valid = pd.concat([valid, final_df[final_df['ID']==i]])

In [49]:
print('Training Set Has: ' + str(len(train.groupby('ID').count())) + ' people')
print('Validation (Test) Set Has: ' + str(len(valid.groupby('ID').count())) + ' people')

Training Set Has: 58 people
Validation (Test) Set Has: 24 people


In [0]:
X_train = train[  ['file_name','Facial Index','Mandibular Index','Intercantal Index','Orbital Width Index','Eye Fissure Index','Vermilion Height Index','Mouth Face Width Index']]
y_train = train[['AGE']]

X_valid = valid[  ['file_name','Facial Index','Mandibular Index','Intercantal Index','Orbital Width Index','Eye Fissure Index','Vermilion Height Index','Mouth Face Width Index']]
y_valid = valid[['AGE']]

In [51]:
X_train.columns

Index(['file_name', 'Facial Index', 'Mandibular Index', 'Intercantal Index',
       'Orbital Width Index', 'Eye Fissure Index', 'Vermilion Height Index',
       'Mouth Face Width Index'],
      dtype='object')

# Pre proocssing X_train & Validation
## keeping only file names in X_train and X_validation

In [55]:
X_train.drop(X_train.columns[[1,2,3,4,5,6,7]], axis = 1, inplace = True) 

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [13]:
X_train.columns

Index(['file_name'], dtype='object')

In [56]:
X_valid.drop(X_valid.columns[[1,2,3,4,5,6,7]], axis = 1, inplace = True) 

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


# Pre processing y_train & Validation
## Using one hot encoding

In [57]:
y_train['AGE']=pd.Categorical(y_train['AGE'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [58]:
dfdummies=pd.get_dummies(y_train['AGE'],prefix='AGE')
y_train=pd.concat([y_train,dfdummies],axis=1)
y_train.shape

(701, 64)

In [0]:
y_train.to_csv('/content/drive/My Drive/y_train.csv')

In [0]:
dfdummies=pd.get_dummies(y_valid['AGE'],prefix='AGE')
y_valid=pd.concat([y_valid,dfdummies],axis=1)


In [0]:
y_valid.drop(y_valid.columns[[0]], axis = 1, inplace = True)
y_train.drop(y_train.columns[[0]], axis = 1, inplace = True)

In [20]:
y_train.dtypes

AGE_0     uint8
AGE_1     uint8
AGE_2     uint8
AGE_3     uint8
AGE_4     uint8
          ...  
AGE_61    uint8
AGE_62    uint8
AGE_63    uint8
AGE_67    uint8
AGE_69    uint8
Length: 63, dtype: object

In [62]:
print(X_train.shape,y_train.shape,X_valid.shape,y_valid.shape)

(701, 1) (701, 63) (301, 1) (301, 47)


# Saving 

In [0]:
X_train.to_csv('/content/drive/My Drive/Aging model/X_train.csv')
y_train.to_csv('/content/drive/My Drive/Aging model/y_train.csv')
X_valid.to_csv('/content/drive/My Drive/Aging model/X_valid.csv')
y_valid.to_csv('/content/drive/My Drive/Aging model/y_valid.csv')

In [0]:
datagen=ImageDataGenerator(rescale=1./255.)
valid_datagen=ImageDataGenerator(rescale=1./255.)

In [59]:
columns=y_train.columns
len(columns)

63

In [0]:
X_train.to_csv('/content/drive/My Drive/X_train.csv')

In [186]:
y_train.head()

,file_name,AGE_0,AGE_1,AGE_2,AGE_3,AGE_4,AGE_5,AGE_6,AGE_7,AGE_8,AGE_9,AGE_10,AGE_11,AGE_12,AGE_13,AGE_14,AGE_15,AGE_16,AGE_17,AGE_18,AGE_19,AGE_20,AGE_21,AGE_22,AGE_23,AGE_24,AGE_25,AGE_26,AGE_27,AGE_28,AGE_29,AGE_30,AGE_31,AGE_32,AGE_33,AGE_34,AGE_35,AGE_36,AGE_37,AGE_38,AGE_39,AGE_40,AGE_41,AGE_42,AGE_43,AGE_44,AGE_45,AGE_46,AGE_47,AGE_48,AGE_49,AGE_50,AGE_51,AGE_52,AGE_53,AGE_54,AGE_55,AGE_58,AGE_60,AGE_61,AGE_62,AGE_63,AGE_67,AGE_69
0,001A02.JPG,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,001A05.JPG,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,001A08.JPG,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,001A10.JPG,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,001A14.JPG,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
X=pd.concat([X_train, y_train])

In [33]:
y_train.columns

Index(['AGE_0', 'AGE_1', 'AGE_2', 'AGE_3', 'AGE_4', 'AGE_5', 'AGE_6', 'AGE_7',
       'AGE_8', 'AGE_9', 'AGE_10', 'AGE_11', 'AGE_12', 'AGE_13', 'AGE_14',
       'AGE_15', 'AGE_16', 'AGE_17', 'AGE_18', 'AGE_19', 'AGE_20', 'AGE_21',
       'AGE_22', 'AGE_23', 'AGE_24', 'AGE_25', 'AGE_26', 'AGE_27', 'AGE_28',
       'AGE_29', 'AGE_30', 'AGE_31', 'AGE_32', 'AGE_33', 'AGE_34', 'AGE_35',
       'AGE_36', 'AGE_37', 'AGE_38', 'AGE_39', 'AGE_40', 'AGE_41', 'AGE_42',
       'AGE_43', 'AGE_44', 'AGE_45', 'AGE_46', 'AGE_47', 'AGE_48', 'AGE_49',
       'AGE_50', 'AGE_51', 'AGE_52', 'AGE_53', 'AGE_54', 'AGE_55', 'AGE_58',
       'AGE_60', 'AGE_61', 'AGE_62', 'AGE_63', 'AGE_67', 'AGE_69'],
      dtype='object')

In [34]:
y_valid.columns

Index(['AGE_0', 'AGE_1', 'AGE_2', 'AGE_3', 'AGE_4', 'AGE_5', 'AGE_6', 'AGE_7',
       'AGE_8', 'AGE_9', 'AGE_10', 'AGE_11', 'AGE_12', 'AGE_13', 'AGE_14',
       'AGE_15', 'AGE_16', 'AGE_17', 'AGE_18', 'AGE_19', 'AGE_20', 'AGE_21',
       'AGE_22', 'AGE_23', 'AGE_24', 'AGE_25', 'AGE_26', 'AGE_27', 'AGE_28',
       'AGE_29', 'AGE_30', 'AGE_31', 'AGE_32', 'AGE_33', 'AGE_34', 'AGE_35',
       'AGE_38', 'AGE_39', 'AGE_40', 'AGE_41', 'AGE_42', 'AGE_43', 'AGE_44',
       'AGE_45', 'AGE_47', 'AGE_53', 'AGE_54'],
      dtype='object')

In [36]:
print(X_train.shape,X_valid.shape,y_train.shape,y_valid.shape)

(701, 1) (301, 1) (701, 63) (301, 47)


In [22]:
X_train.columns

Index(['file_name'], dtype='object')

In [46]:
train_generator=datagen.flow_from_dataframe(
dataframe=X_train,
directory="/content/drive/My Drive/Aging model/fg",
x_col='file_name',
y_col=y_train.columns,
batch_size=32,
seed=42,
shuffle=False,class_mode='input',
target_size=(100,100))

Found 701 validated image filenames.


In [47]:
valid_generator=datagen.flow_from_dataframe(
dataframe=X_valid,
directory="/content/drive/My Drive/Aging model/fg",
x_col='file_name',
y_col=y_valid.columns,
batch_size=32,
seed=42,
shuffle=False,class_mode='input',
target_size=(100,100))

Found 301 validated image filenames.


In [40]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (32, 100, 100, 3)
labels batch shape: (32, 100, 100, 3)


In [0]:
model = Sequential()
model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=(100,100)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), padding='same'))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

other data

In [44]:
df=pd.read_csv('/content/drive/My Drive/miml_dataset/miml_labels_1.csv')
columns=["desert", "mountains", "sea", "sunset", "trees"]
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)
train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory='/content/drive/My Drive/miml_dataset/images',
x_col="Filenames",
y_col=columns,
batch_size=32,
seed=42,
shuffle=True,
class_mode="raw",
target_size=(100,100))

Found 1800 validated image filenames.


In [45]:
for data_batch, labels_batch in train_generator:
  print('data batch shape:', data_batch.shape)
  print('labels batch shape:', labels_batch.shape)
  break

data batch shape: (32, 100, 100, 3)
labels batch shape: (32, 5)


In [0]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)train_generator=datagen.flow_from_dataframe(
dataframe=df[:1800],
directory="./miml_dataset/images",
x_col="Filenames",
y_col="labels",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["desert", "mountains", "sea", "sunset", "trees"],
target_size=(100,100))